In [1]:
import pandas as pd
import torch
from lightning import pytorch as pl
from chemprop import data, models, nn
import json
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import IterableDataset
import numpy as np
import torch
from chemprop import data
import rdkit
from rdkit import Chem
from chemprop import data, featurizers
from chemprop.featurizers.molecule import MorganBinaryFeaturizer
import math
import math
from torch.utils.data import IterableDataset
from chemprop.data.collate import collate_batch
from sklearn.preprocessing import StandardScaler

In [46]:
class Data_Preprocessor:
    '''A class to prepare Chemprop dataset from Pandas dataframe.'''
    
    def __init__(self):
        pass

    
    def is_hbd(self,atom):
        '''Check if an atom is a Hydrogen Bond Donor (HBD). An atom is considered an HBD if it's N or O with at least one hydrogen.
        
        Parameters:
        ----------
        atom: RDKit atom object.
            
        Returns:
        ----------
        bool: True if atom is HBD, False otherwise.
        '''
        
        if atom.GetAtomicNum() not in [7, 8]:  # 7 for N, 8 for O
            return False
        
        n_hydrogens = atom.GetTotalNumHs()
        return n_hydrogens > 0

    
    def is_hba(self,atom):
        '''Check if an atom is a Hydrogen Bond Acceptor (HBA). An atom is considered an HBA if it's N or O with a lone pair electron
        
        Parameters:
        ----------
        atom: RDKit atom object.
            
        Returns:
        ----------
        bool: True if atom is HBD, False otherwise.
        '''
        
        atomic_num = atom.GetAtomicNum()
        if atomic_num not in [7, 8]: 
            return False
        
        valence = atom.GetTotalValence()
        if atomic_num == 7:  
            return valence <= 3 
        else: 
            return valence <= 2  
        

    def get_mol_HBD_HBA(self,mols):
        '''A function to generate HBD_HBA properties for molecules
        
        Parameters:
        ---------
        mols (list): list of RDKit mol objects.
        
        Returns:
        ----------
        mol_HBs (list): list of array that contain HBD-HBA descriptor for molecules, shape of each array is (n_atom, 2)  '''
        mol_HBs = []
        for mol in mols:
            mol_HB = [[],[]]
            for atom in mol.GetAtoms():
                if self.is_hbd(atom):
                    mol_HB[0].append(1)
                else:
                    mol_HB[0].append(0)
                    
                if self.is_hba(atom):
                    mol_HB[1].append(1)
                else:
                    mol_HB[1].append(0)
            mol_HB = np.array(mol_HB).T
            mol_HBs.append(mol_HB)
        return mol_HBs

    

    def dataset_generator(self):
        '''Prepare chemprop dataset without additional HBD/HBA feature.
    
        Returns:
        ----------
        dataset (Chemprop dataset): Chemprop dataset.
        '''
        
        morgan_fp = MorganBinaryFeaturizer()
        def datapoint_generator(df,smiles,y,addH,HB,morgan,weight):
            smis = df.loc[:,smiles].values
            ys = df.loc[:,[y]].values
            mols = [Chem.MolFromSmiles(smi) for smi in smis]

            if weight!= None:
                weights = df.loc[:,weight].values

            if HB:
                mol_HBs = self.get_mol_HBD_HBA(mols)
            else:
                mol_HBs = [None]*len(smis)

            if morgan:
                x_ds = [morgan_fp(mol) for mol in mols]
            else:
                x_ds = [None]*len(smis)
            
            datapoints = [data.MoleculeDatapoint.from_smi(smi,y,add_h=addH, V_f = mol_HB, x_d = x_d, weight=weight) for smi, y, mol_HB, x_d, weight in zip(smis,ys,mol_HBs,x_ds, weights)]
            return datapoints

        datapoints = datapoint_generator(df=self.df,smiles=self.smiles_column,y=self.target_column,addH=self.addH,HB=self.HB,morgan=self.morgan,weight=self.weight_column)
        dataset = data.MoleculeDataset(datapoints, featurizer=self.featurizer)
        return dataset
    

    

    def generate(self, df, smiles_column = 'smiles', target_column='docking_score', addH=False, HB = False, morgan = False, weight_column =None,
                 featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()):
        '''Generate chemprop dataset according to a given configuration

        Parameters:
        ----------
        df (Pandas DataFrame): a data frame that contains SMILES code of compounds.
        smiles_column (str): a string that indicates SMILES column in the data frame.
        target_column (str): a string that indicates the target column (i.e. docking_scores, solubility) in the data frame.
        addH (boolean): to incorporate explicit hydrogen atoms into a molecular graph.
        HB (boolean): to incorporate additional HBD/HBA features for each atom in BatchMolGraph.
        morgan (boolean): to incorporate morgan binaray fingerprint for each molecules
        featurizer (Chemprop Featurizer): a Featurizer from Chemprop to encode features for atoms, bonds, and molecules.
    
        Returns:
        ----------
        dataset (Chemprop dataset): Chemprop dataset
        '''
                     
        self.df = df
        self.smiles_column = smiles_column
        self.target_column = target_column
        self.addH = addH
        self.HB = HB
        self.featurizer = featurizer
        self.morgan = morgan
        self.weight_column = weight_column
        

        return self.dataset_generator()
    

In [41]:
def datapoint_preparator(df,smiles_column,target_column,weight_column):
    smis = df.loc[:,smiles_column].values
    ys = df.loc[:,[target_column]].values
    weights = df.loc[:,weight_column].values
            
    datapoints = [data.MoleculeDatapoint.from_smi(smi,y,weight=weight) for smi, y, weight in zip(smis,ys,weights)]
    return datapoints


def dataset_preparator(df, smiles_column, target_column, weight_column, featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()):
    datapoints = datapoint_preparator(df=df, smiles_column=smiles_column, target_column=target_column,weight_column=weight_column)
    dataset = data.MoleculeDataset(datapoints, featurizer=featurizer)
    return dataset

In [47]:
class IterableMolDatapoints(IterableDataset):
    '''A class to prepare data for streaming, which is a subclass of IterableDataset. 
    The output is a generator that yields one chemprop.data.datasets.Datum at a time.
    '''

    def __init__(self, df, smiles_column, target_column, weight_column, scaler = None, size_at_time=100, shuffle=True):
        '''Parameters:
        ----------
        df (pd.DataFrame): A pandas dataframe containing the data.
        smiles_column (str): The column name containing SMILES strings.
        target_column (str): The column name containing the target values.
        scaler (StandardScaler): A StandardScaler object (already fitted) for normalizing the target values.
        size_at_time (int): The number of samples to transfrom into chemprop.data.datasets.Datum at a time.
        shuffle (boolean): If the df is shuffled.'''
        
        super().__init__()
        self.df = df
        self.smiles_column = smiles_column
        self.target_column = target_column
        self.weight_column = weight_column
        self.size_at_time = size_at_time
        self.shuffle= shuffle
        self.scaler = scaler

    def __len__(self):
        return len(self.df)

    def __iter__(self):
        '''A function to define iteration logic. It take the whole csv data, then shuffled, then access to only a subset of data at a time for transformation.
        The output is a generator that yields chemprop.data.datasets.Datum and ready to put through DataLoader.
        '''

        if self.shuffle:
            df_shuffled = self.df.sample(frac=1).reset_index(drop=True)
        else:
            df_shuffled = self.df.copy()

        # Transform pandas dataframe to molecule dataset according to size_at_time, prevent overloading memory. This is to balance between memory and speed.
        for i in range(0, len(df_shuffled), self.size_at_time):
            df_at_time = df_shuffled.iloc[i:i + self.size_at_time]
            data_generator = Data_Preprocessor()
            df_process = data_generator.generate(df=df_at_time,smiles_column=self.smiles_column,target_column=self.target_column,HB=True,weight_column='weight_lowscores')

            if self.scaler != None: 
                df_process.normalize_targets(scaler = self.scaler)

            # Handling parallelization manually
            worker_info = torch.utils.data.get_worker_info()
            if worker_info is None: 
                for mol in df_process:
                    yield mol
            else: 
                num_workers = worker_info.num_workers
                worker_id = worker_info.id
                for i, mol in enumerate(df_process):
                    if i % num_workers == worker_id:
                        yield mol



In [48]:
data_path = '../DRD2_diverse_data.csv'
smiles_column = 'smiles'
target_column = 'docking_score'
weight_column = 'weight_lowscores'
epochs = 50
batch_size = 64

# Prepare data
df = pd.read_csv(data_path)
df_train = df[df['split_random_1']!='test']
df_val = df[df['split_random_1']=='test']
scaler = StandardScaler().fit(df_train[[target_column]])


train_streaming_dataset = IterableMolDatapoints(
    df=df_train,
    smiles_column=smiles_column,
    target_column=target_column,
    weight_column=weight_column,
    scaler=scaler, shuffle=True, size_at_time=640)

train_loader = data.build_dataloader(
    train_streaming_dataset,
    batch_size=batch_size,
    shuffle=False)

val_streaming_dataset = IterableMolDatapoints(
    df=df_val,
    smiles_column=smiles_column,
    target_column=target_column,
    weight_column=weight_column,
    scaler=scaler, shuffle=False, size_at_time=640)

val_loader = data.build_dataloader(
    val_streaming_dataset,
    batch_size=batch_size,
    shuffle=False)



/tmp/ipykernel_3860846/3902945969.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [57]:
# Establish model
mp = nn.BondMessagePassing(d_v = 74, d_e = 14, d_h = 300,
                           dropout=0.3,
                           depth=5)

agg = nn.NormAggregation(norm=199)
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(n_layers=2,
                       dropout=0.3,
                       input_dim=300,
                       hidden_dim=2200,
                       output_transform=output_transform)
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE(), nn.metrics.R2Score()]

mpnn = models.MPNN(message_passing=mp, 
                   agg = agg, 
                   predictor=ffn, 
                   batch_norm=False, 
                   metrics=metric_list,
                   warmup_epochs=1,
                   init_lr=1.477783789959149e-06,
                   max_lr=0.00012044152141486488,
                   final_lr=0.00011724292252282861)

#mpnn = models.MPNN.load_from_checkpoint('../../../hyperparam_optim_5/best_checkpoint.ckpt')


In [58]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names.*")


checkpointing = ModelCheckpoint(
    "../hyperparam_optim_7/model_7/checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)


trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=epochs,
    callbacks=[checkpointing]
)

trainer.fit(mpnn, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/course/.conda/envs/long_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/course/.conda/envs/long_env/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.

  | Name            | Type               | Params | Mode 
-------------------------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/course/.conda/envs/long_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [8]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=50
)

val_streaming_dataset = IterableMolDatapoints(
    df=df_val,
    smiles_column=smiles_column,
    target_column=target_column,
    weight_column=weight_column,
    scaler=None, shuffle=False, size_at_time=640)

val_loader = data.build_dataloader(
    val_streaming_dataset,
    batch_size=batch_size,
    shuffle=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
model = models.MPNN.load_from_checkpoint('../hyperparam_optim_5/best_checkpoint.ckpt')

/home/course/.conda/envs/long_env/lib/python3.11/site-packages/chemprop/models/model.py:286: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load(checkpoint_path, ma

model

In [11]:
trainer.test(model,val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/course/.conda/envs/long_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/course/.conda/envs/long_env/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker i

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │    0.45445483922958374    │
│          test/r2          │    0.7185391783714294     │
│         test/rmse         │     0.579860270023346     │
└───────────────────────────┴───────────────────────────┘

[{'test/rmse': 0.579860270023346,
  'test/mae': 0.45445483922958374,
  'test/r2': 0.7185391783714294}]